$$
\\\textbf{Question 4:}
\\Convolutional\;neural\;networks\;on\;MNIST
$$

$$
Importing\;the\;neccessary\;libraries:
$$

In [0]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from easydict import EasyDict as edict

$$
Function\;for\;CNN
$$

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

$$
Function\;training\;CNN
$$

In [0]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    train_loss=0
    correct=0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        train_loss += F.nll_loss(output, target, reduction='sum').item()
        loss.backward()
        optimizer.step()
        pred = output.argmax(dim=1, keepdim=True) 
        correct += pred.eq(target.view_as(pred)).sum().item()  
        

    train_loss /= len(train_loader.dataset)
    print('Train Epoch {:.0f}\n Train set: Average loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(epoch,
        train_loss, correct, len(train_loader.dataset),
        100. * correct / len(train_loader.dataset)))
    

$$
Function\;for\;testing\;CNN
$$

In [0]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() 
            pred = output.argmax(dim=1, keepdim=True) 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

$$
Main\;function
$$

In [0]:
def train_eval(Model,batch_size,test_batch_size,epochs,learning_rate,momentum):
    parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
    
    args = edict({
    "batch_size": batch_size,
    "test_batch_size": test_batch_size,
    "epochs":epochs,
    "lr":learning_rate,
    "momentum":momentum,
    "no_cuda":False,
    "seed":1,
    "log_interval":10,
    "save_model":False
    })
    
    use_cuda = not args.no_cuda and torch.cuda.is_available()

    torch.manual_seed(args.seed)

    device = torch.device("cuda" if use_cuda else "cpu")

    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=args.batch_size, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=args.test_batch_size, shuffle=True, **kwargs)


    model = Model.to(device)
    optimizer = optim.SGD(model.parameters()
                          , lr=args.lr, momentum=args.momentum)

    for epoch in range(1, args.epochs + 1):
        train(args, model, device, train_loader, optimizer, epoch)
        test(args, model, device, test_loader)

    if (args.save_model):
        torch.save(model.state_dict(),"mnist_cnn.pt")
        


In [0]:
train_eval(Net()
          ,batch_size=64
          ,test_batch_size=1000
           ,epochs=2
           ,learning_rate=0.01
           ,momentum=0.5)

Train Epoch 1
 Train set: Average loss: 0.3378, Accuracy: 54154/60000 (90.2567%)


Test set: Average loss: 0.1169, Accuracy: 9627/10000 (96.2700%)

Train Epoch 2
 Train set: Average loss: 0.0871, Accuracy: 58399/60000 (97.3317%)


Test set: Average loss: 0.0613, Accuracy: 9811/10000 (98.1100%)



$$
Question\;4(a):
$$

$$
Replacing\;RELUs\;with\;sigmoids\;in\;initial\;layers
$$

In [0]:
class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.sigmoid(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.sigmoid(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [0]:
train_eval(Net1()
          ,batch_size=64
          ,test_batch_size=1000
           ,epochs=2
           ,learning_rate=0.01
           ,momentum=0.5)

Train Epoch 1
 Train set: Average loss: 2.1679, Accuracy: 15845/60000 (26.4083%)


Test set: Average loss: 1.4271, Accuracy: 6776/10000 (67.7600%)

Train Epoch 2
 Train set: Average loss: 0.6720, Accuracy: 49172/60000 (81.9533%)


Test set: Average loss: 0.4156, Accuracy: 8799/10000 (87.9900%)



$$
Obeservations:The\;network\;trained\;with\;Relu\;layers\;performing\;better\;compared\;sigmoid\;layers\;in\;same\;number\;of\;epochs
\\\implies This\;because\;gradients\;backpropagated\;by\;sigmoid\;layers\;are\;very\;less\;compared\;to\;relu\;layers
$$

In [0]:
class Net2(nn.Module):
    def __init__(self,p):
        super(Net2, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)
        self.Dropout=torch.nn.Dropout(p)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.Dropout(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

$$
Training\;different\;CNNs\;with\;different\;dropouts(0.25,0.5,0.75,1)
$$

In [0]:
print("CNN with dropout 0.25")
train_eval(Net2(0.25)
          ,batch_size=64
          ,test_batch_size=1000
           ,epochs=2
           ,learning_rate=0.01
           ,momentum=0.5)
print("CNN with dropout 0.5")
train_eval(Net2(0.5)
          ,batch_size=64
          ,test_batch_size=1000
           ,epochs=2
           ,learning_rate=0.01
           ,momentum=0.5)
print("CNN with dropout 0.75")
train_eval(Net2(0.75)
          ,batch_size=64
          ,test_batch_size=1000
           ,epochs=2
           ,learning_rate=0.01
           ,momentum=0.5)
print("CNN with dropout 1")
train_eval(Net2(1)
          ,batch_size=64
          ,test_batch_size=1000
           ,epochs=2
           ,learning_rate=0.01
           ,momentum=0.5)

CNN with dropout 0.25
Train Epoch 1
 Train set: Average loss: 0.3592, Accuracy: 53566/60000 (89.2767%)


Test set: Average loss: 0.1139, Accuracy: 9639/10000 (96.3900%)

Train Epoch 2
 Train set: Average loss: 0.0991, Accuracy: 58205/60000 (97.0083%)


Test set: Average loss: 0.0668, Accuracy: 9794/10000 (97.9400%)

CNN with dropout 0.5
Train Epoch 1
 Train set: Average loss: 0.3801, Accuracy: 53076/60000 (88.4600%)


Test set: Average loss: 0.1009, Accuracy: 9674/10000 (96.7400%)

Train Epoch 2
 Train set: Average loss: 0.1093, Accuracy: 58032/60000 (96.7200%)


Test set: Average loss: 0.0596, Accuracy: 9806/10000 (98.0600%)

CNN with dropout 0.75
Train Epoch 1
 Train set: Average loss: 0.4582, Accuracy: 51318/60000 (85.5300%)


Test set: Average loss: 0.1041, Accuracy: 9667/10000 (96.6700%)

Train Epoch 2
 Train set: Average loss: 0.1444, Accuracy: 57388/60000 (95.6467%)


Test set: Average loss: 0.0651, Accuracy: 9798/10000 (97.9800%)

CNN with dropout 1
Train Epoch 1
 Train set: Av

$$
Observations:The\;network\;with\;dropouts=0.25,0.5\;acheived\;better\;accuracy\;compared\;to\;networks\;with\;dropouts=0.75,1
\\\implies The\;network\;with\;dropout=1\;is\;predicting\;random\;labels\;because\;gradients\;haven't\;backpropagated\;to\;the\;convolutional\;layers\;and\;network\;is\;not\;trained.
$$

$$
Question\;4(c):
$$

$$
Adding\;batch\;normalization\;layer
$$

In [0]:
class Net3(nn.Module):
    def __init__(self,p):
        super(Net3, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv1_bn = nn.BatchNorm2d(20)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.conv2_bn = nn.BatchNorm2d(50)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)
        self.Dropout=torch.nn.Dropout(p)
       

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = self.conv1_bn(x)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = self.conv2_bn(x)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.Dropout(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

$$
Training\;a\;CNN\;with\;Dropout=0.5\;including\;Batch\;normalization\;layer
$$

In [0]:
train_eval(Net3(0.5)
          ,batch_size=64
          ,test_batch_size=1000
           ,epochs=2
           ,learning_rate=0.01
           ,momentum=0.5)

Train Epoch 1
 Train set: Average loss: 0.2251, Accuracy: 56461/60000 (94.1017%)


Test set: Average loss: 0.0588, Accuracy: 9823/10000 (98.2300%)

Train Epoch 2
 Train set: Average loss: 0.0679, Accuracy: 58815/60000 (98.0250%)


Test set: Average loss: 0.0425, Accuracy: 9860/10000 (98.6000%)



$$
Observations:The\;model\;is\;acheiving\;better\;accuracy\;when\;we\;add\;batch\;normalization\;layers
$$

$$
Above\;Network\;after\;removing\;Dropout
$$

In [0]:
class Net4(nn.Module):
    def __init__(self,):
        super(Net4, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv1_bn = nn.BatchNorm2d(20)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.conv2_bn = nn.BatchNorm2d(50)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)
       

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = self.conv1_bn(x)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = self.conv2_bn(x)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [0]:
train_eval(Net4()
          ,batch_size=64
          ,test_batch_size=1000
           ,epochs=2
           ,learning_rate=0.01
           ,momentum=0.5)

Train Epoch 1
 Train set: Average loss: 0.1927, Accuracy: 57219/60000 (95.3650%)


Test set: Average loss: 0.0602, Accuracy: 9818/10000 (98.1800%)

Train Epoch 2
 Train set: Average loss: 0.0542, Accuracy: 59073/60000 (98.4550%)


Test set: Average loss: 0.0466, Accuracy: 9861/10000 (98.6100%)



$$
Observations:There\;is\;no\;considerable\;change\;in\;accuracy\;if\;we\;remove\;dropout\;from\;the\;network\;with\;batchnormalization\;layers
\\\implies We\;don't\;need\;dropout\;if\;the\;network\;has\;batch\;normalization\;layers
$$

$$
Initializing\;the\;CNN\;weights\;using\;xavier\;initialization:
$$

In [0]:
class Net5(nn.Module):
    def __init__(self,):
        super(Net5, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        torch.nn.init.xavier_uniform_(self.conv1.weight)
        self.conv1_bn = nn.BatchNorm2d(20)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        torch.nn.init.xavier_uniform_(self.conv2.weight)
        self.conv2_bn = nn.BatchNorm2d(50)
        self.fc1 = nn.Linear(4*4*50, 500)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.fc2 = nn.Linear(500, 10)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
       

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = self.conv1_bn(x)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = self.conv2_bn(x)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [0]:
train_eval(Net5()
          ,batch_size=64
          ,test_batch_size=1000
           ,epochs=2
           ,learning_rate=0.01
           ,momentum=0.5)

Train Epoch 1
 Train set: Average loss: 0.1254, Accuracy: 57878/60000 (96.4633%)


Test set: Average loss: 0.0540, Accuracy: 9839/10000 (98.3900%)

Train Epoch 2
 Train set: Average loss: 0.0443, Accuracy: 59243/60000 (98.7383%)


Test set: Average loss: 0.0437, Accuracy: 9855/10000 (98.5500%)



$$
Observations:The\;Accuracy\;on\;network\;with\;batch\;normalization\;layers\;haven't\;changed\;with\;different\;weight\;initializations.
\\\implies Different\;weight\;initializations\;dont\;effect\;the\;performance\;of\;the\;network\;with\;batch\;normalizations.
\\\implies Because\;the\;batch\;normalization\;layers\;achieve\;optimal\;mean\;and\;varaince\;of\;the\;layers\;activations\;irrespective\;of\;weight\;initializations
$$